# Arch and Freq Invariance

This notebook shows the architecture and cpu frequency invariance features of EAS.

In [1]:
import logging
reload(logging)
logging.basicConfig(
    format='%(asctime)-9s %(levelname)-8s: %(message)s',
    datefmt='%I:%M:%S')

# Enable logging at INFO level
logging.getLogger().setLevel(logging.INFO)
# Comment the follwing line to disable devlib debugging statements
# logging.getLogger('ssh').setLevel(logging.DEBUG)

In [2]:
# Generate plots inline
%pylab inline

import json
import os

import subprocess

# Support to access the remote target
import devlib
from env import TestEnv

# Support for trace events analysis
from trace import Trace
#from trace_analysis import TraceAnalysis

# Support to configure and run RTApp based workloads
from wlgen import RTA

# Support for performance analysis of RTApp workloads
from perf_analysis import PerfAnalysis

# Suport for FTrace events parsing and visualization
import trappy

Populating the interactive namespace from numpy and matplotlib


# Test environment setup

In [5]:
# Setup a target configuration
androidsdk_path=os.path.join(os.getcwd(), "../../../android-sdk")
my_target_conf = {
    
    "platform"    : 'android',
    "board"       : "hikey960",
    "device" : "0123456789ABCDEF",
    
    "ANDROID_HOME" : androidsdk_path,
    "rtapp-calib" : {"0": 302, "1": 302, "2": 304, "3": 304, "4": 136, "5": 137, "6": 136, "7": 136},

     "emeter" : {
        "instrument" : "acme",
        "conf" : {
            # Absolute path to the iio-capture binary on the host
            'iio-capture' : '/usr/bin/iio-capture',
            # Default host name of the BeagleBone Black
            'ip_address'     : '10.169.36.36',
        },
        "channel_map" : {
            "Device0" : 0, # iio:device0
            "Device1" : 1, # iio:device0
        }
    },
    
    # Define devlib module to load
    #"modules"     : [
    #    'bl',           # enable big.LITTLE support
    #    'cpufreq'       # enable CPUFreq support
    #],

    # Account to access the remote target
    #"host"        : '192.168.1.101',
    "host"        : '10.169.36.74',
    "username"    : 'root',
    "password"    : '',

    # Comment the following line to force rt-app calibration on your target
#     "rtapp-calib" : {
#         '0': 361, '1': 138, '2': 138, '3': 352, '4': 360, '5': 353
#     }
}

# Setup the required Test Environment supports
my_tests_conf = {
    
    # Binary tools required to run this experiment
    # These tools must be present in the tools/ folder for the architecture
    "tools"   : ['rt-app', 'taskset', 'trace-cmd'],
    
    # FTrace events end buffer configuration
    "ftrace"  : {
         "events" : [
            "cpu_frequency",
            "sched_load_avg_cpu",
            "sched_load_avg_task",
            "sched_switch",
            'dequeue_task_fair', 
            'enqueue_task_fair', 
            'set_next_entity',

                    
            "cpu_capacity",
            
            #/sys/kernel/debug/tracing/events/sched
            "sched_blocked_reason",
            "sched_boost_cpu",
            "sched_boost_task",
            "sched_contrib_scale_f",
            "sched_cpu_hotplug",
            "sched_energy_diff",
            "sched_kthread_stop",
            "sched_kthread_stop_ret",
            "sched_load_avg_cpu",
            "sched_load_avg_task",
            "sched_migrate_task",
            "sched_move_numa",
            "sched_pi_setprio",
            "sched_process_exec",
            "sched_process_exit",
            "sched_process_fork",
            "sched_process_free",
            "sched_process_wait",
            "sched_stat_blocked",
            "sched_stat_iowait",
            "sched_stat_runtime",
            "sched_stat_sleep",
            "sched_stat_wait",
            "sched_stick_numa",
            "sched_swap_numa",
            "sched_switch",
            "sched_tune_boostgroup_update",
            "sched_tune_config",
            "sched_tune_tasks_update",
            "sched_tune_filter",
            "sched_wait_task",
            "sched_wake_idle_without_ipi",
            "sched_wakeup",
            "sched_wakeup_new",
         ],
         "buffsize" : 10240
    },

    "results_dir" : "arch_and_freq_invariance",
}

In [6]:
# Support to access the remote target
import devlib
from env import TestEnv

# Initialize a test environment using:
# the provided target configuration (my_target_conf)
# the provided test configuration   (my_test_conf)
te = TestEnv(target_conf=my_target_conf, test_conf=my_tests_conf)
target = te.target

02:14:15  INFO    : Using base path: /home/steven/lisa/lisa-github
02:14:15  INFO    : Loading custom (inline) target configuration
02:14:15  INFO    : Loading custom (inline) test configuration
02:14:15  WARNING : Wipe previous contents of the results folder:
02:14:15  WARNING :    /home/steven/lisa/lisa-github/results/arch_and_freq_invariance
02:14:15  INFO    : External tools using:
02:14:15  INFO    :    ANDROID_HOME: /home/steven/lisa/lisa-github/ipynb/workshop/../../../android-sdk
02:14:15  INFO    :    CATAPULT_HOME: /home/steven/lisa/lisa-github/tools/catapult
02:14:15  INFO    : Devlib modules to load: ['bl', 'cpuidle', 'cpufreq']
02:14:15  INFO    : Connecting Android target [0123456789ABCDEF]
02:14:15  INFO    : Connection settings:
02:14:15  INFO    :    {'device': '0123456789ABCDEF'}
02:14:19  INFO    : Initializing target workdir:
02:14:19  INFO    :    /data/local/tmp/devlib-target
02:14:25  INFO    : Attempting to read energy model from target
02:14:28  INFO    : Topolo

# Workload configuration

In [17]:
# Support to configure and run RTApp based workloads
from wlgen import RTA, Periodic, Ramp

# Create a new RTApp workload generator using the calibration values
# reported by the TestEnv module
rtapp = RTA(target, 'simple', calibration=te.calibration())

# Configure this RTApp instance to:
rtapp.conf(
    # 1. generate a "profile based" set of tasks
    kind='profile',
    
    # 2. define the "profile" of each task
    params={
        
        
        'task_p50': Periodic(
            period_ms=400,         # period
            duty_cycle_pct=50,     # duty cycle
            duration_s=10,         # duration    
            cpus=str(target.bl.bigs[0])   # pinned on first big CPU
        ).get(),
        
        'task_r20': Ramp(
            start_pct=5,           # intial load
            end_pct=95,            # end load
            delta_pct=10,          # load % increase...
            time_s=1,              # ... every 1[s]            
        ).get(),
    },
    
    # 4. use this folder for task logfiles
    run_dir=target.working_directory
    
);

01:43:47  INFO    : Setup new workload simple
01:43:47  INFO    : Workload duration defined by longest task
01:43:47  INFO    : Default policy: SCHED_OTHER
01:43:47  INFO    : ------------------------
01:43:47  INFO    : task [task_p50], sched: using default policy
01:43:47  INFO    :  | loops count: 1
01:43:47  INFO    :  | CPUs affinity: 4
01:43:47  INFO    : + phase_000001: duration 10.000000 [s] (25 loops)
01:43:47  INFO    : |  period   400000 [us], duty_cycle  50 %
01:43:47  INFO    : |  run_time 200000 [us], sleep_time 200000 [us]
01:43:47  INFO    : ------------------------
01:43:47  INFO    : task [task_r20], sched: using default policy
01:43:47  INFO    :  | loops count: 1
01:43:47  INFO    : + phase_000001: duration 1.000000 [s] (10 loops)
01:43:47  INFO    : |  period   100000 [us], duty_cycle   5 %
01:43:47  INFO    : |  run_time   5000 [us], sleep_time  95000 [us]
01:43:47  INFO    : + phase_000002: duration 1.000000 [s] (10 loops)
01:43:47  INFO    : |  period   100000 [

# Workload execution round 1

In [18]:
# Set performance governor
#
# For Juno big/LITTLE CPUs
#
logging.info("Target ABI: %s, CPus: %s",
             target.abi,
             target.cpuinfo.cpu_names)

target.cpufreq.set_all_governors('performance')

logging.info("Target current governor: %s",
             target.read_value("/sys/devices/system/cpu/cpu0/cpufreq/scaling_governor")
            )
logging.info("Target LITTLE CPU max CPUfreq:%s\n\t\t    Target LITTLE CPU current CPUfreq: %s", 
             target.read_value("/sys/devices/system/cpu/cpu0/cpufreq/scaling_max_freq"),
             target.read_value("/sys/devices/system/cpu/cpu0/cpufreq/scaling_cur_freq")
             )

logging.info("Target big CPU max CPUfreq:%s \n\t\t    Target big CPU current CPUfreq: %s", 
             target.read_value("/sys/devices/system/cpu/cpu4/cpufreq/scaling_max_freq"),
             target.read_value("/sys/devices/system/cpu/cpu4/cpufreq/scaling_cur_freq")
             )

01:43:49  INFO    : Target ABI: arm64, CPus: ['A53', 'A53', 'A53', 'A53', 'A73', 'A73', 'A73', 'A73']
01:43:49  INFO    : Target current governor: performance
01:43:50  INFO    : Target LITTLE CPU max CPUfreq:1844000
		    Target LITTLE CPU current CPUfreq: 1844000
01:43:50  INFO    : Target big CPU max CPUfreq:2362000 
		    Target big CPU current CPUfreq: 2362000


In [19]:
logging.info('#### Setup FTrace')
te.ftrace.start()

logging.info('#### Start energy sampling')
te.emeter.reset()

logging.info('#### Start RTApp execution')
rtapp.run(out_dir=te.res_dir, cgroup="")

logging.info('#### Read energy consumption: %s/energy.json', te.res_dir)
nrg_report = te.emeter.report(out_dir=te.res_dir)

logging.info('#### Stop FTrace')
te.ftrace.stop()

trace_file = os.path.join(te.res_dir, 'trace.dat')
logging.info('#### Save FTrace: %s', trace_file)
te.ftrace.get_trace(trace_file)

logging.info('#### Save platform description: %s/platform.json', te.res_dir)
(plt, plt_file) = te.platform_dump(te.res_dir)

01:43:51  INFO    : #### Setup FTrace
01:43:54  INFO    : #### Start energy sampling
01:43:55  INFO    : #### Start RTApp execution
01:43:55  INFO    : Workload execution START:
01:43:55  INFO    :    /data/local/tmp/bin/rt-app /data/local/tmp/devlib-target/simple_00.json 2>&1
01:44:05  INFO    : #### Read energy consumption: /home/steven/lisa/lisa-github/results/arch_and_freq_invariance/energy.json
01:44:05  ERROR   : /usr/bin/iio-capture terminated for Device1 (iio:device1)
01:44:05  ERROR   : [Unable to create IIO context
]
01:44:05  ERROR   : /usr/bin/iio-capture terminated for Device0 (iio:device0)
01:44:05  ERROR   : [Unable to create IIO context
]
01:44:05  INFO    : #### Stop FTrace
01:44:05  INFO    : #### Save FTrace: /home/steven/lisa/lisa-github/results/arch_and_freq_invariance/trace.dat
01:44:07  INFO    : #### Save platform description: /home/steven/lisa/lisa-github/results/arch_and_freq_invariance/platform.json


# Collected results

In [20]:
# All data are produced in the output folder defined by the TestEnv module
logging.info('Content of the output folder %s', te.res_dir)
!ls -la {te.res_dir}

01:44:07  INFO    : Content of the output folder /home/steven/lisa/lisa-github/results/arch_and_freq_invariance


total 7504
drwxrwxr-x 1 steven steven     284 12月 22 13:44 .
drwxrwxr-x 1 steven steven    1552 12月 22 13:43 ..
-rw-rw-r-- 1 steven steven       2 12月 22 13:44 energy.json
-rw-rw-r-- 1 steven steven       2 12月 22 13:44 energy_stats.json
-rw-rw-r-- 1 steven steven     307 12月 22 13:44 output.log
-rw-rw-r-- 1 steven steven    1526 12月 22 13:44 platform.json
-rw-r--r-- 1 steven steven    3260 12月 22 13:44 rt-app-task_p50-0.log
-rw-r--r-- 1 steven steven   12560 12月 22 13:44 rt-app-task_r20-1.log
-rw-r--r-- 1 steven steven    5120 12月 22 13:44 rt-app-task_r20_5-60-1.log
-rw-r--r-- 1 steven steven    3265 12月 22 13:44 simple_00.json
-rw-r--r-- 1 steven steven 7634944 12月 22 13:44 trace.dat


In [21]:
# Inspect the JSON file used to run the application
with open('{}/simple_00.json'.format(te.res_dir), 'r') as fh:
    rtapp_json = json.load(fh, )
logging.info('Generated RTApp JSON file:')
#print json.dumps(rtapp_json, indent=4, sort_keys=True)

01:44:07  INFO    : Generated RTApp JSON file:


In [22]:
# Dump the energy measured for the LITTLE and big clusters
logging.info('Energy: %s', nrg_report.report_file)
print json.dumps(nrg_report.channels, indent=4, sort_keys=True)

01:44:08  INFO    : Energy: /home/steven/lisa/lisa-github/results/arch_and_freq_invariance/energy.json


{}


In [23]:
# Dump the platform descriptor, which could be useful for further analysis
# of the generated results
logging.info('Platform description: %s', plt_file)
#print json.dumps(plt, indent=4, sort_keys=True)

01:44:08  INFO    : Platform description: /home/steven/lisa/lisa-github/results/arch_and_freq_invariance/platform.json


# Trace inspection

In [24]:
# Suport for FTrace events parsing and visualization
import trappy

# NOTE: The interactive trace visualization is available only if you run
#       the workload to generate a new trace-file
trappy.plotter.plot_trace(te.res_dir)

In [25]:
events_to_parse = my_tests_conf['ftrace']['events']

trace = Trace(plt, te.res_dir, events_to_parse)

ftrace = trace.ftrace

trappy.plotter.plot_trace(ftrace, execnames=['task_p50', "task_r20" ])

/home/steven/lisa/lisa-github/libs/trappy/trappy/ftrace.py:274: UserWarning: TRAPpy: Couldn't read <trappy.dynamic.set_next_entity object at 0x7fd8e89d7b10> from cache, reading it from trace
  warnings.warn(warnstr)
/home/steven/lisa/lisa-github/libs/trappy/trappy/ftrace.py:274: UserWarning: TRAPpy: Couldn't read <trappy.dynamic.cpu_frequency_devlib object at 0x7fd8e89d7c50> from cache, reading it from trace
  warnings.warn(warnstr)
/home/steven/lisa/lisa-github/libs/trappy/trappy/ftrace.py:274: UserWarning: TRAPpy: Couldn't read <trappy.dynamic.dequeue_task_fair object at 0x7fd8e89d7e50> from cache, reading it from trace
  warnings.warn(warnstr)
/home/steven/lisa/lisa-github/libs/trappy/trappy/ftrace.py:274: UserWarning: TRAPpy: Couldn't read <trappy.dynamic.sched_wait_task object at 0x7fd8e89d7f50> from cache, reading it from trace
  warnings.warn(warnstr)
/home/steven/lisa/lisa-github/libs/trappy/trappy/ftrace.py:274: UserWarning: TRAPpy: Couldn't read <trappy.dynamic.sched_stat_blo

# Workload Execution Round 2

In [26]:
# Set userspace governor with lowest CPUfreq
#
# For Juno big/LITTLE CPUs
#
logging.info("Target ABI: %s, CPus: %s",
             target.abi,
             target.cpuinfo.cpu_names)

target.cpufreq.set_all_governors('userspace')

LITTLE_min_freq=target.read_value("/sys/devices/system/cpu/cpu0/cpufreq/scaling_min_freq")
big_min_freq=target.read_value("/sys/devices/system/cpu/cpu4/cpufreq/scaling_min_freq")


target.write_value("/sys/devices/system/cpu/cpu0/cpufreq/scaling_setspeed", LITTLE_min_freq)
target.write_value("/sys/devices/system/cpu/cpu4/cpufreq/scaling_setspeed", big_min_freq)

logging.info("Target current governor: %s",
             target.read_value("/sys/devices/system/cpu/cpu0/cpufreq/scaling_governor")
            )
logging.info("Target big CPU min CPUfreq:%s\n\t\t    Target big CPU current CPUfreq: %s", 
             target.read_value("/sys/devices/system/cpu/cpu4/cpufreq/scaling_min_freq"),
             target.read_value("/sys/devices/system/cpu/cpu4/cpufreq/scaling_cur_freq")
             )

logging.info("Target LITTLE CPU min CPUfreq:%s \n\t\t    Target LITTLE CPU current CPUfreq: %s", 
             target.read_value("/sys/devices/system/cpu/cpu0/cpufreq/scaling_min_freq"),
             target.read_value("/sys/devices/system/cpu/cpu0/cpufreq/scaling_cur_freq")
             )

01:44:17  INFO    : Target ABI: arm64, CPus: ['A53', 'A53', 'A53', 'A53', 'A73', 'A73', 'A73', 'A73']
01:44:18  INFO    : Target current governor: userspace
01:44:18  INFO    : Target big CPU min CPUfreq:903000
		    Target big CPU current CPUfreq: 903000
01:44:18  INFO    : Target LITTLE CPU min CPUfreq:533000 
		    Target LITTLE CPU current CPUfreq: 533000


In [27]:
logging.info('#### Setup FTrace')
te.ftrace.start()

logging.info('#### Start energy sampling')
te.emeter.reset()

logging.info('#### Start RTApp execution')
rtapp.run(out_dir=te.res_dir, cgroup="")

logging.info('#### Read energy consumption: %s/energy.json', te.res_dir)
nrg_report = te.emeter.report(out_dir=te.res_dir)

logging.info('#### Stop FTrace')
te.ftrace.stop()

trace_file = os.path.join(te.res_dir, 'trace.dat')
logging.info('#### Save FTrace: %s', trace_file)
te.ftrace.get_trace(trace_file)

logging.info('#### Save platform description: %s/platform.json', te.res_dir)
(plt, plt_file) = te.platform_dump(te.res_dir)

01:44:18  INFO    : #### Setup FTrace
01:44:22  INFO    : #### Start energy sampling
01:44:23  INFO    : #### Start RTApp execution
01:44:23  INFO    : Workload execution START:
01:44:23  INFO    :    /data/local/tmp/bin/rt-app /data/local/tmp/devlib-target/simple_00.json 2>&1
01:45:04  INFO    : #### Read energy consumption: /home/steven/lisa/lisa-github/results/arch_and_freq_invariance/energy.json
01:45:04  ERROR   : /usr/bin/iio-capture terminated for Device1 (iio:device1)
01:45:04  ERROR   : [Unable to create IIO context
]
01:45:04  ERROR   : /usr/bin/iio-capture terminated for Device0 (iio:device0)
01:45:04  ERROR   : [Unable to create IIO context
]
01:45:04  INFO    : #### Stop FTrace
01:45:05  INFO    : #### Save FTrace: /home/steven/lisa/lisa-github/results/arch_and_freq_invariance/trace.dat
01:45:10  INFO    : #### Save platform description: /home/steven/lisa/lisa-github/results/arch_and_freq_invariance/platform.json


In [28]:
# All data are produced in the output folder defined by the TestEnv module
logging.info('Content of the output folder %s', te.res_dir)
!ls -la {te.res_dir}

01:45:10  INFO    : Content of the output folder /home/steven/lisa/lisa-github/results/arch_and_freq_invariance


total 12648
drwxrwxr-x 1 steven steven      316 12月 22 13:45 .
drwxrwxr-x 1 steven steven     1552 12月 22 13:43 ..
-rw-rw-r-- 1 steven steven        2 12月 22 13:45 energy.json
-rw-rw-r-- 1 steven steven        2 12月 22 13:45 energy_stats.json
-rw-rw-r-- 1 steven steven      307 12月 22 13:45 output.log
-rw-rw-r-- 1 steven steven     1526 12月 22 13:45 platform.json
-rw-r--r-- 1 steven steven     3260 12月 22 13:45 rt-app-task_p50-0.log
-rw-r--r-- 1 steven steven    12560 12月 22 13:45 rt-app-task_r20-1.log
-rw-r--r-- 1 steven steven     5120 12月 22 13:45 rt-app-task_r20_5-60-1.log
-rw-r--r-- 1 steven steven     3265 12月 22 13:45 simple_00.json
-rw-r--r-- 1 steven steven 12902400 12月 22 13:45 trace.dat
drwxrwxr-x 1 steven steven     2664 12月 22 13:44 .trace.dat.cache


# Collected Results

# Trace inspection

In [29]:
# Suport for FTrace events parsing and visualization
import trappy

# NOTE: The interactive trace visualization is available only if you run
#       the workload to generate a new trace-file
trappy.plotter.plot_trace(te.res_dir)

/home/steven/lisa/lisa-github/libs/trappy/trappy/ftrace.py:201: UserWarning: Cached data is from another trace, invalidating cache.
  warnings.warn(warnstr)


In [30]:
events_to_parse = my_tests_conf['ftrace']['events']

trace = Trace(plt, te.res_dir, events_to_parse)

ftrace = trace.ftrace

trappy.plotter.plot_trace(ftrace, execnames=['task_p50', "task_r20" ])

/home/steven/lisa/lisa-github/libs/trappy/trappy/ftrace.py:274: UserWarning: TRAPpy: Couldn't read <trappy.dynamic.set_next_entity object at 0x7fd8e857cd10> from cache, reading it from trace
  warnings.warn(warnstr)
/home/steven/lisa/lisa-github/libs/trappy/trappy/ftrace.py:274: UserWarning: TRAPpy: Couldn't read <trappy.dynamic.cpu_frequency_devlib object at 0x7fd8e857ce50> from cache, reading it from trace
  warnings.warn(warnstr)
/home/steven/lisa/lisa-github/libs/trappy/trappy/ftrace.py:274: UserWarning: TRAPpy: Couldn't read <trappy.dynamic.dequeue_task_fair object at 0x7fd8e8558090> from cache, reading it from trace
  warnings.warn(warnstr)
/home/steven/lisa/lisa-github/libs/trappy/trappy/ftrace.py:274: UserWarning: TRAPpy: Couldn't read <trappy.dynamic.sched_wait_task object at 0x7fd8e8558190> from cache, reading it from trace
  warnings.warn(warnstr)
/home/steven/lisa/lisa-github/libs/trappy/trappy/ftrace.py:274: UserWarning: TRAPpy: Couldn't read <trappy.dynamic.sched_stat_blo